In [1]:
import os
import sys 
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
from datetime import timedelta

import time
import pytz
import math
import locale

import openpyxl
from openpyxl import load_workbook

# import hvplot.pandas
# import plotly.graph_objects as go
# import matplotlib.pyplot as plt

import talib
from pybit import HTTP
# import requests 
# import json 

#pd.show_versions()

### Set environment variables from the .env in the local environment

In [2]:
load_dotenv()
my_api_key = os.getenv("API_KEY")
my_api_secret = os.getenv("API_SECRET")

# print(my_api_key)
# print(my_api_secret)

### Change Timezone if Needed

In [3]:
## my_list simply unpacks the list elements and pass each one of them as parameters to the print function  
# print(*pytz.all_timezones, sep='\n')

# In windows command prompt try:
#     This gives current timezone: tzutil /g
#     This gives a list of timezones: tzutil /l
#     This will set the timezone: tzutil /s "Central America Standard Time"


#os.system('tzutil /s "Eastern Standard Time"')  
#os.system('tzutil /s "Singapore Standard Time"')
#time.strftime('%Y-%m-%d %H:%m') 

### Functions Used to Pull Data from ByBit and Write the Data to Files

In [4]:
# Adjust from_time to include prior 200 entries for that interval for ema200
def adjust_from_time(from_time, interval):
    
    delta = 200
    
    # Possible Values: 1 3 5 15 30 60 120 240 360 720 "D" "W" 
    if interval not in [1, 3, 5, 15, 30, 60, 120, 240, 360, 720, "D", "W"]:
        return from_time
    
    if interval == 'W':
        from_time = from_time - timedelta(weeks=delta)
    elif interval == 'D':
        from_time = from_time - timedelta(days=delta)
    else:
        from_time = from_time - timedelta(minutes=interval*delta)
    return from_time
        
def build_filename(params, extension):
    from_str = params['FROM_TIME'].strftime('%Y-%m-%d')
    to_str = params['TO_TIME'].strftime('%Y-%m-%d')
    filename = f'{params["SYMBOL"]}_{from_str}_to_{to_str}_{params["INTERVAL"]}{extension}'
    return filename

def get_bybit_kline_data(params):
    
    # Unauthenticated
    #session_unauth = HTTP(endpoint='https://api.bybit.com')

    # Authenticated
    session_auth = HTTP(
        endpoint = 'https://api.bybit.com',
        api_key = my_api_key,
        api_secret = my_api_secret
    )
    
    # The issue with ByBit API is that you can get a maximum of 200 bars from it. 
    # So if you need to get data for a large portion of the time you have to call it multiple times.

    df_list = []
    adjusted_from_time = adjust_from_time(params['FROM_TIME'], params['INTERVAL'])
    last_datetime = adjusted_from_time
    print(f'Fetching data from ByBit.')
    
    while last_datetime < params['TO_TIME']:
        #print(f'Fetching next 200 lines fromTime: [{last_datetime}]')
        last_datetime_timestamp = str(int(last_datetime.timestamp()))
        result = session_auth.query_kline(symbol=params['SYMBOL'], interval=params['INTERVAL'], **{'from':last_datetime_timestamp})['result']
        tmp_df = pd.DataFrame(result)

        if tmp_df is None or (len(tmp_df.index) == 0):
            break
            
        tmp_df.index = [dt.datetime.fromtimestamp(x) for x in tmp_df.open_time]
        df_list.append(tmp_df)
        last_datetime = max(tmp_df.index) + dt.timedelta(0, 1) # Add 1s to last data received
        
        #time.sleep(2) # Sleep for x seconds, to avoid being locked out
 
    #if len(df_list) > 0:
    df = pd.concat(df_list)

    # Drop rows that have a timestamp greater than to_time
    df = df[df.open_time <= int(params['TO_TIME'].timestamp())]
    
    # Write to file
    if 'csv' in params['OUPUT_FILE_FORMAT']:
        fname = params['HISTORICAL_FILES_PATH'] + '\\' + build_filename(params, '.csv')
        df.to_csv (fname, index = True, header=True)
        print(f'Data written to file => [{fname}]')
        
    if 'xlsx' in params['OUPUT_FILE_FORMAT']:
        fname = params['HISTORICAL_FILES_PATH'] + '\\' + build_filename(params, '.xlsx')
        df.to_excel (fname, index = True, header=True)
        print(f'Data written to file => [{fname}]')
    
    return df

def convertExcelToDataFrame(filename):
    wb = load_workbook(filename)
    ws = wb['Sheet1']

    # To convert a worksheet to a Dataframe you can use the values property. 
    # This is very easy if the worksheet has no headers or indices:
    # df = DataFrame(ws.values)

    # https://openpyxl.readthedocs.io/en/stable/pandas.html

    # If the worksheet does have headers or indices, such as one created by Pandas, 
    # then a little more work is required:
    from itertools import islice
    data = ws.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df = pd.DataFrame(data, index=idx, columns=cols)

    return df

### Calculate for Indicators and Signals

In [5]:
# ----------------------------------------------------------------------
# Calculate and add indicators and signals to the DataFrame
# ----------------------------------------------------------------------
def add_indicators_and_signals(df):
    print('Adding indicators and Signals to Data.')
    
    # Set proper data types
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(np.int64)
    
    # Keep only this list of columns, delete all other columns
    final_table_columns = ['symbol', 'interval', 'open', 'high', 'low', 'close', 'volume']
    df = df[df.columns.intersection(final_table_columns)]
    #df.filter(final_table_columns)
    
    ## MACD - Moving Average Convergence/Divergence
    tmp = pd.DataFrame()
    tmp['macd'], tmp['macdsignal'], tmp['macdhist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    tmp.drop(['macdhist'], axis = 1, inplace=True)
    df = df.join(tmp, rsuffix='_right')

    ## EMA - Exponential Moving Average
    df['ema200'] = talib.EMA(df['close'], timeperiod=200)

    # # Remove nulls
    df.dropna(inplace=True)

    # # Check if price is greater than ema200
    df['GT_ema200'] = np.where(df['open'] > df['ema200'], 'Bull', 'Bear')

    # macdsignal over macd then 1, under 0
    df['signal_over_under'] = np.where(df['macdsignal'] >= df['macd'], 1, 0)

    # macdsignal crosses macd
    df['cross'] = df['signal_over_under'].diff()

    # Remove nulls
    df.dropna(inplace=True)

    # Enter Trade
    df['trade_status'] = df.apply(lambda x: trade_entries(x['open'], x['ema200'], x['macdsignal'], x['cross']), axis=1)

    # Add and Initialize to 0 new columns for take_profit/stop_loss and profits/loss
    df['take_profit'] = 0.0
    df['stop_loss'] = 0.0
    df['win'] = 0.0
    df['loss'] = 0.0
    df['fee'] = 0.0

    return df

### Based on Indicators Create Trades and Statistics

In [6]:
# ----------------------------------------------------------------------
# Print Statistics
# ----------------------------------------------------------------------
def print_trade_stats(total_wins, total_losses, nb_wins, nb_losses, total_fees_paid, max_conseq_longs, max_conseq_shorts):
    total_trades = nb_wins + nb_losses
    success_rate = (nb_wins / total_trades * 100) if total_trades != 0 else 0
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
    print(f'\nWinning Trades: {nb_wins}')
    print(f'Max # Consecutive Longs: {max_conseq_longs}')
    print(f'Losing Trades: {nb_losses}')
    print(f'Max # Consecutive Shorts: {max_conseq_shorts}')
    print(f'Total Trades: {total_trades}')
    print(f'Success Rate: {success_rate:.1f}%')
    print()
    print(f'Total Wins: {locale.currency(total_wins, grouping=True)}');
    print(f'Total Losses: {locale.currency(total_losses, grouping=True)}');
    print(f'Total Fees Paid: {locale.currency(total_fees_paid, grouping=True)}')
    print(f'Total P/L: {locale.currency(total_wins+total_losses-total_fees_paid, grouping=True)}')
    
# Returns 2 values. 
# 1) Maximum number of consecutive long trades within the date range
# 2) Maximum number of consecutive Short trades within the date range
def get_max_occurences_long_short(df):
    df2 = df[df['trade_status'].notnull() & df['trade_status'].str.contains('Enter')]
    df2 = df2.filter(['trade_status'])
    df2['trade_status'] = df2['trade_status'].replace(['Enter Long', 'Enter/Exit Long'], 'L')
    df2['trade_status'] = df2['trade_status'].replace(['Enter Short', 'Enter/Exit Short'], 'S')
    values = df2['trade_status'].values.tolist()
    
    last_index = len(values)-1
    count_L = 0
    count_S = 0
    max_L = 0
    max_S = 0

    for i, val in enumerate(values):
        if val == 'L':
            count_L += 1
            if i == last_index and count_L > max_L:
                max_L = count_L
            elif i < last_index and values[i+1] != val:
                    if count_L > max_L:
                        max_L = count_L
                    count_L = 0
        elif val == 'S':
            count_S += 1
            if i == last_index and count_S > max_S:
                max_S = count_S
            elif i < last_index and values[i+1] != val:
                    if count_S > max_S:
                        max_S = count_S
                    count_S = 0
        #print(f'Index[{i}] Value[{val}] - count_L: {count_L}, count_S: {count_S}, max_L: {max_L}, max_S: {max_S}')
    return max_L, max_S

# ----------------------------------------------------------------------
# Function used determine trade entries (long/short)
# ----------------------------------------------------------------------
def trade_entries(open, ema200, macdsignal, cross):
    if open >= ema200 and macdsignal < 0 and cross == -1:
        return "Enter Long"
    elif open < ema200 and macdsignal > 0 and cross == 1:
        return "Enter Short"
    return None

# ----------------------------------------------------------------------------
# Process Trades: Add Trades to the Dataframe, write results to Excel files
# ----------------------------------------------------------------------------
def process_trades(df, params):
    print('Processing Trades.')
    nb_wins = 0
    nb_losses = 0

    position_size = 0.0
    entry_price = 0.0
    stop_loss = 0.0
    take_profit = 0.0

    trade_status = ''
    total_wins = 0.0
    total_losses = 0.0
    total_fees_paid = 0.0

    # We use numeric indexing to update values in the DataFrame
    # Find the column indexes
    trade_status_col_index = df.columns.get_loc("trade_status")
    tp_col_index = df.columns.get_loc("take_profit")
    sl_col_index = df.columns.get_loc("stop_loss")
    wins_col_index = df.columns.get_loc("win")
    losses_col_index = df.columns.get_loc("loss")
    fee_col_index = df.columns.get_loc("fee")

    TP_PCT = params['TAKE_PROFIT_PCT'] / 100
    SL_PCT = params['STOP_LOSS_PCT'] / 100
    FEE_PCT = params['FEES_PCT'] / 100

    for i, row in enumerate(df.itertuples(index=True), 0):

        # ------------------------------- Longs -------------------------------
        if trade_status == '' and row.trade_status == 'Enter Long':
            
            entry_price = row.open
            stop_loss = entry_price - (SL_PCT * entry_price)
            take_profit = entry_price + (TP_PCT * entry_price)
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss
            # Entry Fee
            entry_fee = params['TRADE_AMOUNT'] * FEE_PCT
            df.iloc[i, fee_col_index] += entry_fee
            total_fees_paid += entry_fee
            
            # We exit in the same candle we entered, hit stop loss
            if row.low <= stop_loss:
                loss = params['TRADE_AMOUNT'] * SL_PCT * -1
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Long'
                df.iloc[i, losses_col_index] = loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                #Exit Fee 'loss'
                exit_fee = (params['TRADE_AMOUNT'] - loss) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
                
            # We exit in the same candle we entered, take profit
            elif row.high >= take_profit:
                win = params['TRADE_AMOUNT'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Long'
                df.iloc[i, wins_col_index] = win
                total_wins += win
                trade_status = ''
                nb_wins += 1
                #Exit Fee 'win'
                exit_fee = (params['TRADE_AMOUNT'] + win) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
                
            # We just entered 'Enter long' in this candle so set the status to 'Long'
            else:
                trade_status = 'Long'

        elif trade_status in ['Long'] and pd.isnull(row.trade_status):
            if row.low <= stop_loss:
                loss = params['TRADE_AMOUNT'] * SL_PCT * -1
                df.iloc[i, trade_status_col_index] = 'Exit Long'
                df.iloc[i, losses_col_index] = loss
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                # Exit Fee 'loss'
                exit_fee = (params['TRADE_AMOUNT'] - loss) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            elif row.high >= take_profit:
                win = params['TRADE_AMOUNT'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Exit Long'
                df.iloc[i, wins_col_index] = win
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_wins += win
                trade_status = ''
                nb_wins += 1
                # Exit Fee 'win'
                exit_fee = (params['TRADE_AMOUNT'] + win) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            else:
                df.iloc[i, trade_status_col_index] = 'Long'
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                trade_status = 'Long'

        elif trade_status in ['Long'] and row.trade_status in ['Enter Long', 'Enter Short']:
            # If we are in a long and encounter another 'Enter Long' or a 'Enter Short' signal,
            # ignore the signal and override the value with 'Long', we are already in a 'Long' trade
            df.iloc[i, trade_status_col_index] = 'Long'
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss

        # ------------------------------- Shorts ------------------------------- 
        elif trade_status == '' and row.trade_status == 'Enter Short':
            entry_price = row.open
            #position_size = (params['TRADE_AMOUNT'] / row.open) if row.open != 0 else 0
            stop_loss = entry_price + (SL_PCT * entry_price)
            take_profit = entry_price - (TP_PCT * entry_price)
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss
            # Entry Fee
            entry_fee = params['TRADE_AMOUNT'] * FEE_PCT
            df.iloc[i, fee_col_index] += entry_fee
            total_fees_paid += entry_fee
                
             # We exit in the same candle we entered, hit stop loss
            if row.high >= stop_loss:
                loss = SL_PCT * params['TRADE_AMOUNT'] * -1
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Short'
                df.iloc[i, losses_col_index] = loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                # Exit Fee 'loss'
                exit_fee = (params['TRADE_AMOUNT'] + loss) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
             # We exit in the same candle we entered, hit take profit
            elif row.low <= take_profit:
                win = params['TRADE_AMOUNT'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Enter/Exit Short'
                df.iloc[i, wins_col_index] = win
                total_wins += win
                trade_status = ''
                nb_wins += 1
                # Exit Fee 'loss'
                exit_fee = (params['TRADE_AMOUNT'] - win) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            # We just entered 'Enter Short' in this candle, so set the status to 'Short'
            else:
                trade_status = 'Short'

        elif trade_status in ['Short'] and pd.isnull(row.trade_status):
            if row.high >= stop_loss:
                loss = SL_PCT * params['TRADE_AMOUNT'] * -1
                df.iloc[i, trade_status_col_index] = 'Exit Short'
                df.iloc[i, losses_col_index] = loss
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_losses += loss
                trade_status = ''
                nb_losses += 1
                # Exit Fee 'loss'
                exit_fee = (params['TRADE_AMOUNT'] + loss) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            elif row.low <= take_profit:
                win = params['TRADE_AMOUNT'] * TP_PCT
                df.iloc[i, trade_status_col_index] = 'Exit Short'
                df.iloc[i, wins_col_index] = win
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                total_wins += win
                trade_status = ''
                nb_wins += 1
                # Exit Fee 'win'
                exit_fee = (params['TRADE_AMOUNT'] - win) * FEE_PCT
                df.iloc[i, fee_col_index] += exit_fee
                total_fees_paid += exit_fee
            else:
                df.iloc[i, trade_status_col_index] = 'Short'
                df.iloc[i, tp_col_index] = take_profit
                df.iloc[i, sl_col_index] = stop_loss
                trade_status = 'Short'

        elif trade_status in ['Short'] and row.trade_status in ['Enter Long', 'Enter Short']:
            # If we are in a long and encounter another 'Enter Long' or a 'Enter Short' signal,
            # ignore the signal and override the value with 'Long', we are already in a 'Short' trade
            df.iloc[i, trade_status_col_index] = 'Short'
            df.iloc[i, tp_col_index] = take_profit
            df.iloc[i, sl_col_index] = stop_loss


    if 'csv' in params['OUPUT_FILE_FORMAT']:
        fname = params['RESULTS_PATH'] + '\\' + build_filename(params, '_Trades.csv')
        df.to_csv (fname, index = True, header=True)
        print(f'File created containing trade details => [{fname}]')

    if 'xlsx' in params['OUPUT_FILE_FORMAT']:
        fname = params['RESULTS_PATH'] + '\\' + build_filename(params, '_Trades.xlsx')
        df.to_excel (fname, index = True, header=True)
        print(f'File created containing trade details => [{fname}]')
        
    max_conseq_longs, max_conseq_shorts = get_max_occurences_long_short(df)
    
    print_trade_stats(total_wins, total_losses, nb_wins, nb_losses, total_fees_paid, max_conseq_longs, max_conseq_shorts)

    return df

In [7]:
# Print parameter values. 
# 'all'=True prints all values
# 'all'=False prints only relevant ones (default)
def print_parameters(params, all=False):
    if all:
        for key, value in params.items():
            print(f'{key}: {value}')
        print()
    else:
        #print("Parameter Values:")
        print(f'SYMBOL: {params["SYMBOL"]}')
        print(f'FROM_TIME: {params["FROM_TIME"]}')
        print(f'TO_TIME: {params["TO_TIME"]}')
        print(f'INTERVAL: {params["INTERVAL"]}')
        print(f'TRADE_AMOUNT: {params["TRADE_AMOUNT"]}')
        print(f'TAKE_PROFIT_PCT: {params["TAKE_PROFIT_PCT"]}%')
        print(f'STOP_LOSS_PCT: {params["STOP_LOSS_PCT"]}%')
        print(f'FEES_PCT: {params["FEES_PCT"]}%\n')
    
def validate_params(params):
    if not isinstance(params["FROM_TIME"], dt.datetime):
        raise Exception(f'Invalid Parameter [FROM_TIME] = {params["FROM_TIME"]}')
        
    if not isinstance(params["TO_TIME"], dt.datetime):
        raise Exception(f'Invalid Parameter [TO_TIME] = {params["TO_TIME"]}')
        
    if params["INTERVAL"] not in [1, 3, 5, 15, 30, 60, 120, 240, 360, 720, "D", "W"]:
        raise Exception(f'Invalid Parameter [INTERVAL] = {params["INTERVAL"]}')
        
    trade_amount = params["TRADE_AMOUNT"]
    if not isinstance(trade_amount, float) or trade_amount <= 0:
        raise Exception(f'Invalid Parameter [TRADE_AMOUNT] = {trade_amount}. Must be a positive value of type float.')
        
    take_profit_pct = params["TAKE_PROFIT_PCT"]
    if not isinstance(take_profit_pct, float) or take_profit_pct <= 0:
        raise Exception(f'Invalid Parameter [TAKE_PROFIT_PCT] = {take_profit_pct}. Must be a positive value of type float.')
        
    stop_loss_pct = params["STOP_LOSS_PCT"]
    if not isinstance(stop_loss_pct, float) or stop_loss_pct <= 0:
        raise Exception(f'Invalid Parameter [STOP_LOSS_PCT] = {stop_loss_pct}. Must be a positive value of type float.')
        
    fees_pct = params["FEES_PCT"]
    if not isinstance(fees_pct, float) or fees_pct <= 0:
        raise Exception(f'Invalid Parameter [FEES_PCT] = {fees_pct}. Must be a positive value of type float.')
    
    # Convert all items to lower case
    formats_list = params["OUPUT_FILE_FORMAT"]
    if not isinstance(formats_list, list) or len(params['OUPUT_FILE_FORMAT']) == 0:
        raise Exception(f'Invalid Parameter [OUPUT_FILE_FORMAT] = {formats_list}.')
    params['OUPUT_FILE_FORMAT'] = [x.lower() for x in params['OUPUT_FILE_FORMAT']]
    
    
def backtest(params):
    
    print_parameters(params)
    validate_params(params)

    # Method 1 (slow): Get historical data directly from the ByBit API
    # --------------------------------------------------------------------
    df = get_bybit_kline_data (params)
    if df is None:
        print(f'\nNo data was returned from ByBit. Unable to backtest strategy.')
        raise Exception("No data returned by ByBit")
    elif len(df) <= params['MIN_DATA_SIZE']:
        print(f'\nData rows = {len(df)}, less than MIN_DATA_SIZE={params["MIN_DATA_SIZE"]}. Unable to backtest strategy.')
        raise Exception("Unable to Run Strategy on Data Set")

    # Method 2 (fast): Get historical data from previously saved files
    # --------------------------------------------------------------------
    # filename = 'ByBit Historical\\BTCUSDT_2021-01-01_to_2021-11-27_30.xlsx'
    # print(f'Reading data from file => [{filename}]')
    # df = convertExcelToDataFrame(filename)

    df = add_indicators_and_signals(df);
    df = process_trades(df, params)
    return df

### Running the BackTesting

In [8]:
# Dictionary containing backtesting parameters
params = {
    'SYMBOL' : 'BTCUSDT'
    , 'FROM_TIME' : dt.datetime(2020, 3, 1) # (year, month, day, hour, minutes, seconds)
    , 'TO_TIME' : dt.datetime(2021, 11, 30)
    , 'INTERVAL' : 30 # Possible Values: 1 3 5 15 30 60 120 240 360 720 "D" "W"
    , 'TRADE_AMOUNT' : float(10000) # Amount used per trade. Remains constant throughout time
    , 'TAKE_PROFIT_PCT' : float(2) # 2%
    , 'STOP_LOSS_PCT' : float(1)   # 1%
    , 'FEES_PCT' : float(0.075) # 0.075%
    
    , 'MIN_DATA_SIZE' : 201 # Cannot run Strategy on data set less than this value
    , 'HISTORICAL_FILES_PATH' : 'ByBit Historical' # Folder location where to store ByBit original raw data
    , 'RESULTS_PATH' : 'BackTesting Results' # Folder location where to store the back testing output
    , 'OUPUT_FILE_FORMAT' : ['xlsx']  # Prefered format(s) for the ouput: csv, xlsx or both
}

# try:
#     print_parameters()
#     backtest(SYMBOL, INTERVAL, FROM_TIME, TO_TIME, TRADE_AMOUNT, TAKE_PROFIT_PCT, STOP_LOSS_PCT)
# except Exception as e:
#     print('Program Terminated: '+repr(e))
 
# Run backtesting
df = backtest(params)

SYMBOL: BTCUSDT
FROM_TIME: 2020-03-01 00:00:00
TO_TIME: 2021-11-30 00:00:00
INTERVAL: 30
TRADE_AMOUNT: 10000.0
TAKE_PROFIT_PCT: 2.0%
STOP_LOSS_PCT: 1.0%
FEES_PCT: 0.075%

Fetching data from ByBit.
Data written to file => [ByBit Historical\BTCUSDT_2020-03-01_to_2021-11-30_30.xlsx]
Adding indicators and Signals to Data.
Processing Trades.
File created containing trade details => [BackTesting Results\BTCUSDT_2020-03-01_to_2021-11-30_30_Trades.xlsx]

Winning Trades: 155
Max # Consecutive Longs: 17
Losing Trades: 166
Max # Consecutive Shorts: 10
Total Trades: 321
Success Rate: 48.3%

Total Wins: $31,000.00
Total Losses: ($16,600.00)
Total Fees Paid: $4,821.90
Total P/L: $9,578.10
